In [1]:
import requests  
import pandas as pd
import numpy as np
from datetime import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [4]:
def fetch_worldbank_data(indicator_code, indicator_name):
    """Fetch data from World Bank API"""
    url = f"http://api.worldbank.org/v2/country/USA/indicator/{indicator_code}?format=json&per_page=100"
    response = requests.get(url)
    data = response.json()[1]  # Actual data is in the second element
    
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date']).dt.date
    df = df[['date', 'value']].rename(columns={'value': indicator_name})
    df = df.sort_values('date')
    return df

# Fetch all macroeconomic indicators
gdp = fetch_worldbank_data("NY.GDP.MKTP.CD", "gdp")
interest_rate = fetch_worldbank_data("FR.INR.RINR", "real_interest_rate")
unemployment = fetch_worldbank_data("SL.UEM.TOTL.NE.ZS", "unemployment_rate")
inflation = fetch_worldbank_data("FP.CPI.TOTL.ZG", "inflation_rate")

print(gdp)
print(interest_rate)
print(unemployment)
print(inflation)

          date           gdp
63  1960-01-01  5.419886e+11
62  1961-01-01  5.619403e+11
61  1962-01-01  6.036394e+11
60  1963-01-01  6.370586e+11
59  1964-01-01  6.841446e+11
..         ...           ...
4   2019-01-01  2.153998e+13
3   2020-01-01  2.135410e+13
2   2021-01-01  2.368117e+13
1   2022-01-01  2.600689e+13
0   2023-01-01  2.772071e+13

[64 rows x 2 columns]
          date  real_interest_rate
63  1960-01-01                 NaN
62  1961-01-01            3.107885
61  1962-01-01            3.215346
60  1963-01-01            3.374882
59  1964-01-01            2.951669
..         ...                 ...
4   2019-01-01            3.573062
3   2020-01-01            2.186282
2   2021-01-01           -1.258522
1   2022-01-01                 NaN
0   2023-01-01                 NaN

[64 rows x 2 columns]
          date  unemployment_rate
63  1960-01-01              5.500
62  1961-01-01              6.700
61  1962-01-01              5.500
60  1963-01-01              5.700
59  1964-01-01  

In [7]:
def process_macro_data(gdp, interest, unemployment, inflation):
    """Combine and process macroeconomic data"""
    # Merge all data
    macro_df = gdp.merge(interest, on='date', how='outer')
    macro_df = macro_df.merge(unemployment, on='date', how='outer')
    macro_df = macro_df.merge(inflation, on='date', how='outer')
    
    # Convert to datetime and set as index 
    macro_df['date'] = pd.to_datetime(macro_df['date'])
    macro_df = macro_df.set_index('date')
    
    # Forward fill quarterly data to daily
    macro_df = macro_df.resample('D').ffill()
    
    # Calculate growth rates and moving averages
    macro_df['gdp_growth'] = macro_df['gdp'].pct_change(periods=90)  # Quarterly growth
    macro_df['unemployment_ma'] = macro_df['unemployment_rate'].rolling(90).mean()
    macro_df['inflation_ma'] = macro_df['inflation_rate'].rolling(90).mean()
    
    # Fill any remaining NA values
    macro_df = macro_df.fillna(method='bfill')
    
    return macro_df.reset_index()

macro_data = process_macro_data(gdp, interest_rate, unemployment, inflation)

print(macro_data.head())
print ("macro data download finished!")

        date           gdp  real_interest_rate  unemployment_rate  \
0 1960-01-01  5.419886e+11            3.107885                5.5   
1 1960-01-02  5.419886e+11            3.107885                5.5   
2 1960-01-03  5.419886e+11            3.107885                5.5   
3 1960-01-04  5.419886e+11            3.107885                5.5   
4 1960-01-05  5.419886e+11            3.107885                5.5   

   inflation_rate  gdp_growth  unemployment_ma  inflation_ma  
0        1.457976         0.0              5.5      1.457976  
1        1.457976         0.0              5.5      1.457976  
2        1.457976         0.0              5.5      1.457976  
3        1.457976         0.0              5.5      1.457976  
4        1.457976         0.0              5.5      1.457976  
macro data download finished!


C:\Users\liyua\AppData\Local\Temp\ipykernel_13148\515161342.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  macro_df = macro_df.fillna(method='bfill')
